# Importing H2o library 

In [3]:
import h2o

# Requesting Connection

In [4]:
h2o.init(nthreads = -1, max_mem_size = 8)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11.0.2" 2019-01-15 LTS; Java(TM) SE Runtime Environment 18.9 (build 11.0.2+9-LTS); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.2+9-LTS, mixed mode)
  Starting server from /Users/pavantej/opt/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/ss/2w2rzn7j48ldjn3q4y2h55pc0000gn/T/tmp4c5y1v9d
  JVM stdout: /var/folders/ss/2w2rzn7j48ldjn3q4y2h55pc0000gn/T/tmp4c5y1v9d/h2o_pavantej_started_from_python.out
  JVM stderr: /var/folders/ss/2w2rzn7j48ldjn3q4y2h55pc0000gn/T/tmp4c5y1v9d/h2o_pavantej_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.3
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_pavantej_nbba2q
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


# Loading Ratings file 

In [5]:
rating_csv='/Users/pavantej/Desktop/SCIT/sem2/big data/20180701_Batch39_CSE7322c_Recommendation/ml-latest-small/rating_edx.csv'

In [8]:
data = h2o.import_file(rating_csv)

H2OResponseError: Server error water.exceptions.H2ONotFoundArgumentException:
  Error: File /Users/pavantej/Desktop/SCIT/sem2/big data/20180701_Batch39_CSE7322c_Recommendation/ml-latest-small/rating_edx.csv does not exist
  Request: POST /3/ImportFilesMulti
    data: {'paths': '[/Users/pavantej/Desktop/SCIT/sem2/big data/20180701_Batch39_CSE7322c_Recommendation/ml-latest-small/rating_edx.csv]'}


# shape of Ratings file 

In [7]:
data.shape

NameError: name 'data' is not defined

# loading Movies data

In [36]:
movies_csv='/Users/pavantej/Desktop/SCIT/sem2/big data/20180701_Batch39_CSE7322c_Recommendation/ml-latest-small/movies.csv'

In [37]:
data_movies=h2o.import_file(movies_csv)

Parse progress: |█████████████████████████████████████████████████████████| 100%


# shape of movies data

In [38]:
data_movies.shape

(27278, 3)

# splitting the data into train and test and validation 

In [39]:
splits = data.split_frame(ratios=[0.8, 0.1], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]

# Number of counts 

In [40]:
print(train.nrow)
print (valid.nrow)
print (test.nrow)

839086
104407
105082


In [213]:
y = 'rating'
x = list(data.columns)

In [214]:
x

['userId', 'movieId', 'rating', 'timestamp']

In [215]:
y

'rating'

In [216]:
x.remove(y)

In [217]:
x

['userId', 'movieId', 'timestamp']

In [218]:
x.remove('timestamp')

In [219]:
x

['userId', 'movieId']

# Building Random Forest 1

In [220]:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [221]:
rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1)

In [222]:
rf_fit1.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [51]:
rf_perf1 = rf_fit1.model_performance(test)

# Building Random Forest 2

In [52]:
rf_fit2 = H2ORandomForestEstimator(model_id='rf_fit2', ntrees=500, seed=1)
rf_fit2.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [54]:
rf_perf2=rf_fit2.model_performance(test)

# Performance Metrics of both the Random Forest

In [55]:
print (rf_perf1.auc)
print(rf_perf2.auc)


ModelMetricsRegression: drf
** Reported on test data. **

MSE: 0.9464477519400134
RMSE: 0.9728554630262469
MAE: 0.7652385515882485
RMSLE: 0.26052885007121324
Mean Residual Deviance: 0.9464477519400134
<bound method MetricsBase.auc of >

ModelMetricsRegression: drf
** Reported on test data. **

MSE: 0.942661674274772
RMSE: 0.9709076548646488
MAE: 0.7638785709621067
RMSLE: 0.2601718297830711
Mean Residual Deviance: 0.942661674274772
<bound method MetricsBase.auc of >


# Predicting on the test data using the best possible Random forest 

In [56]:
preds_test_rf=rf_fit2.predict(test)

drf prediction progress: |████████████████████████████████████████████████| 100%


In [57]:
preds_test_rf

predict
4.25851
4.00653
4.12271
4.10105
3.7599
3.79877
3.94652
3.47525
4.02296
3.89948


In [58]:
preds_test_rf.nrows

105082

In [59]:
test['Preds_RF']=preds_test_rf

In [60]:
test

userId,movieId,rating,timestamp,Preds_RF
1,260,4,1.11248e+09,4.25851
1,924,3.5,1.09479e+09,4.00653
1,1222,3.5,1.11248e+09,4.12271
1,1304,3,1.09479e+09,4.10105
1,1350,3.5,1.09479e+09,3.7599
1,1848,3.5,1.11249e+09,3.79877
1,2143,4,1.11249e+09,3.94652
1,2683,3.5,1.09479e+09,3.47525
1,2947,3.5,1.11249e+09,4.02296
1,3499,4,1.11248e+09,3.89948


# Gradient Boosting Machine

In [61]:
#Import H2O GBM:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

# Training using a default GBM

In [62]:
# Initialize and train the GBM estimator:

gbm_fit1 = H2OGradientBoostingEstimator(model_id='gbm_fit1', seed=1)
gbm_fit1.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


# Training  using  GBM with more trees


In [63]:
gbm_fit2 = H2OGradientBoostingEstimator(model_id='gbm_fit2', ntrees=500, seed=1)
gbm_fit2.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


# Training using  GBM with early stopping

In [64]:
# Now let's use early stopping to find optimal ntrees

gbm_fit3 = H2OGradientBoostingEstimator(model_id='gbm_fit3', 
                                        ntrees=500, 
                                        score_tree_interval=5,     #used for early stopping
                                        stopping_rounds=3,         #used for early stopping
                                        #stopping_metric='AUC',     #used for early stopping
                                        stopping_tolerance=0.0005, #used for early stopping
                                        seed=1)

# The use of a validation_frame is recommended with using early stopping
gbm_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


# Compare model performance

In [65]:
gbm_perf1 = gbm_fit1.model_performance(test)
gbm_perf2 = gbm_fit2.model_performance(test)
gbm_perf3 = gbm_fit3.model_performance(test)

In [66]:
# Retreive test set AUC
print (gbm_perf1.auc)
print (gbm_perf2.auc)
print (gbm_perf3.auc)


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 1.0362073481362184
RMSE: 1.0179427037590172
MAE: 0.8081807887497252
RMSLE: 0.2720639720390326
Mean Residual Deviance: 1.0362073481362184
<bound method MetricsBase.auc of >

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 1.018970108808796
RMSE: 1.0094404929508207
MAE: 0.8004098437406081
RMSLE: 0.2700838540732006
Mean Residual Deviance: 1.018970108808796
<bound method MetricsBase.auc of >

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 1.018970108808796
RMSE: 1.0094404929508207
MAE: 0.8004098437406081
RMSLE: 0.2700838540732006
Mean Residual Deviance: 1.018970108808796
<bound method MetricsBase.auc of >


# predicting using the best GBM 

In [67]:
preds_GBM=gbm_fit2.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [68]:
preds_GBM

predict
3.97722
4.06259
4.11316
4.00862
3.65534
3.4555
3.62012
3.54708
3.74987
3.66349


In [69]:
test['preds_GBM']=preds_GBM

In [75]:
test.show

userId,movieId,rating,timestamp,Preds_RF,preds_GBM
1,260,4,1.11248e+09,4.25851,3.97722
1,924,3.5,1.09479e+09,4.00653,4.06259
1,1222,3.5,1.11248e+09,4.12271,4.11316
1,1304,3,1.09479e+09,4.10105,4.00862
1,1350,3.5,1.09479e+09,3.7599,3.65534
1,1848,3.5,1.11249e+09,3.79877,3.4555
1,2143,4,1.11249e+09,3.94652,3.62012
1,2683,3.5,1.09479e+09,3.47525,3.54708
1,2947,3.5,1.11249e+09,4.02296,3.74987
1,3499,4,1.11248e+09,3.89948,3.66349


<bound method H2OFrame.show of >

In [76]:
test.shape

(105082, 6)

# Deep Learning

In [78]:
# Import H2O DL:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [92]:
# Initialize and train the DL estimator:

dl_fit2 = H2ODeepLearningEstimator(model_id='dl_fit2', 
                                   epochs=20, 
                                   hidden=[5,5], 
                                   stopping_rounds=0,  #disable early stopping
                                   seed=1)
dl_fit2.train(x=x, y=y, training_frame=train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [104]:
dl_fit3 = H2ODeepLearningEstimator(model_id='dl_fit3', 
                                   epochs=20, 
                                   hidden=[10,10],
                                   score_interval=1,          #used for early stopping
                                   stopping_rounds=3,         #used for early stopping
                                   #stopping_metric='AUC',     #used for early stopping
                                   stopping_tolerance=0.0005, #used for early stopping
                                   seed=1)
dl_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [105]:
dl_perf2 = dl_fit2.model_performance(test)
dl_perf3 = dl_fit3.model_performance(test)

In [106]:
print (dl_perf2.auc)
print (dl_perf3.auc)


ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 1.10577015941525
RMSE: 1.0515560657498249
MAE: 0.8401877569562359
RMSLE: 0.28031377811888614
Mean Residual Deviance: 1.10577015941525
<bound method MetricsBase.auc of >

ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 1.1082407619179437
RMSE: 1.05273014676979
MAE: 0.8422151592904912
RMSLE: 0.2799999736465173
Mean Residual Deviance: 1.1082407619179437
<bound method MetricsBase.auc of >


In [107]:
preds_NN=dl_fit3.predict(test)

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [108]:
preds_NN

predict
3.53627
3.51476
3.50511
3.50245
3.50096
3.48483
3.47527
3.45778
3.44923
3.43879


In [109]:
test['preds_NN']=preds_NN

In [110]:
test

userId,movieId,rating,timestamp,Preds_RF,preds_GBM,preds_NN
1,260,4,1.11248e+09,4.25851,3.97722,3.53627
1,924,3.5,1.09479e+09,4.00653,4.06259,3.51476
1,1222,3.5,1.11248e+09,4.12271,4.11316,3.50511
1,1304,3,1.09479e+09,4.10105,4.00862,3.50245
1,1350,3.5,1.09479e+09,3.7599,3.65534,3.50096
1,1848,3.5,1.11249e+09,3.79877,3.4555,3.48483
1,2143,4,1.11249e+09,3.94652,3.62012,3.47527
1,2683,3.5,1.09479e+09,3.47525,3.54708,3.45778
1,2947,3.5,1.11249e+09,4.02296,3.74987,3.44923
1,3499,4,1.11248e+09,3.89948,3.66349,3.43879


In [182]:
WA=((0.5*test['Preds_RF'])+(0.3*test['preds_GBM'])+(0.2*test['preds_NN']))

In [183]:
WA

Preds_RF
4.02967
3.92499
3.99632
3.9536
3.67674
3.633
3.75435
3.4933
3.82629
3.73654


In [185]:
Avg=(test['Preds_RF']+test['preds_GBM']+test['preds_NN'])/3

In [186]:
Avg

Preds_RF
3.924
3.86129
3.91366
3.87071
3.63873
3.5797
3.68064
3.49337
3.74069
3.66725


In [200]:
test['Avg']=Avg

This H2OFrame has been removed.


<bound method H2OFrame.sum of >